In [5]:
import copy
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import utils

import warnings; warnings.simplefilter('ignore')

from mpl_toolkits.mplot3d import Axes3D
from nilearn import datasets, plotting
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Low Dimensional Connectome Dynamics

### Prepare Data - Separate Modalities

In [ ]:
all_subjects_all_trials_connectomes = utils.load_connectomes(utils.ALL_SUBJECT_IDS, utils.ALL_TRIAL_IDS)
all_subjects_all_trials_connectomes['fmri'].shape

Extract flattened representation of upper triangular of Pearson correlation matrix for each connectome type.

In [2]:
# NOTE: The below logic would have to change if we move away from using Desikan Atlas where the number of regions 
# are the same between EEG and fMRI
num_regions = all_subjects_all_trials_connectomes['fmri'].shape[1]
num_regions

NameError: name 'all_subjects_all_trials_connectomes' is not defined

In [3]:
upper_triangular_including_diagonal_idxs = np.triu_indices(num_regions, k=0)
lower_triangular_idxs = np.tril_indices(num_regions, k=-1)

NameError: name 'np' is not defined

In [4]:
all_subjects_all_trials_connectome_upper_triangular_flattened = copy.deepcopy(all_subjects_all_trials_connectomes)
for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    all_subjects_all_trials_connectome_upper_triangular_flattened[k] = np.array([c[upper_triangular_including_diagonal_idxs].flatten() for c in all_subjects_all_trials_connectomes[k]])

NameError: name 'copy' is not defined

In [ ]:
all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape

## Principal Component Analysis

Compute principal components of every connectome time series individually.

In [ ]:
n_components = 10
pca_models = [PCA(n_components=n_components).fit(all_subjects_all_trials_connectome_upper_triangular_flattened[k]) for k in all_subjects_all_trials_connectome_upper_triangular_flattened]

Plot variance explained ratio for each connectome type.

In [ ]:
fig = plt.figure(figsize=(30, 5))
fig.suptitle('Total Variance Explained w/ ' + str(n_components) + ' Components')

subplot_idx = 1
for (k, pca_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, pca_models):
    fig.add_subplot(1, len(pca_models), subplot_idx)
    plt.plot(pca_model.explained_variance_ratio_)
    plt.title("{0} - {1:.2f}%".format(k, 100*sum(pca_model.explained_variance_ratio_)))
    subplot_idx += 1    

plt.savefig('output/principal_component_analysis/variance_explained.png')
fig.close()

Plot spatial representation of each principal component in connectome space.

In [1]:
pca_models[0].components_.shape

NameError: name 'pca_models' is not defined

In [ ]:
fig = plt.figure(figsize=(180, 60))
fig.suptitle('Spatial Loadings of Principal Components', fontsize=40)

subplot_idx = 1
for (k, pca_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, pca_models):
    for component_idx in range(0, n_components):
        
        # Extract connectome representation of the principal component
        pc = np.zeros((num_regions, num_regions))
        pc[upper_triangular_including_diagonal_idxs] = pca_model.components_[component_idx]
        pc[lower_triangular_idxs] = pc.T[lower_triangular_idxs]

        # Plot connectome representation of the principal component
        ax = fig.add_subplot(len(pca_models), n_components, subplot_idx)
        plotting.plot_connectome(pc, desikan_atlas_coordinates(), title='{0} PC{1} Connectome'.format(k, component_idx+1),
                                 edge_threshold='95%', node_size=20, colorbar=True, axes=ax)
        subplot_idx += 1

plt.savefig('output/principal_component_analysis/spatial_loadings.png')
fig.close()

Plot correlation of principal component with connectome through time.

In [ ]:
def add_arrow(line, position=None, direction='right', size=15, color=None):
    """
    add an arrow to a line.

    line:       Line2D object
    position:   x-position of the arrow. If None, mean of xdata is taken
    direction:  'left' or 'right'
    size:       size of the arrow in fontsize points
    color:      if None, line color is taken.
    """
    if color is None:
        color = line.get_color()

    xdata = line.get_xdata()
    ydata = line.get_ydata()

    if position is None:
        position = xdata.mean()
    # find closest index
    start_ind = np.argmin(np.absolute(xdata - position))
    if direction == 'right':
        end_ind = start_ind + 1
    else:
        end_ind = start_ind - 1

    line.axes.annotate('',
        xytext=(xdata[start_ind], ydata[start_ind]),
        xy=(xdata[end_ind], ydata[end_ind]),
        arrowprops=dict(arrowstyle="->", color=color),
        size=size
    )

In [ ]:
fig = plt.figure(figsize=(300, 300))
fig.suptitle('Low-Dimensional Manifold Traversed by Brain State', fontsize=60)

subplot_idx = 1
for (k, pca_model) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, pca_models):
    
    correlation_timeseries_for_first_three_pcs = []
    
    # Gather correlation timeseries for the first 3 principal components
    for component_idx in range(0, 3):
        
        # Compute correlation between pc and connectome timeseries
        pc = pca_model.components_[component_idx]
        connectome_timeseries = all_subjects_all_trials_connectome_upper_triangular_flattened[k]
        
        correlation_between_pc_and_connectome_timeseries = [np.corrcoef(pc, time_pt)[0, 1] for time_pt in connectome_timeseries]
        correlation_timeseries_for_first_three_pcs.append(correlation_between_pc_and_connectome_timeseries)
        
    # Cluster time series
    time_series_data = np.array(correlation_timeseries_for_first_three_pcs)
    time = np.arange(0, 3*time_series_data[0].shape[0], step=3)
    time_series_data_including_time = np.vstack((time_series_data, time)).T
    clustered_correlation_timeseries_for_first_three_pcs = KMeans(n_clusters=30).fit(time_series_data_including_time)
    time_ordered_cluster_centers = clustered_correlation_timeseries_for_first_three_pcs.cluster_centers_
    time_ordered_cluster_centers = time_ordered_cluster_centers[time_ordered_cluster_centers[:, 3].argsort()]
    
    ax = fig.add_subplot(len(pca_models), 5, subplot_idx, projection='3d')
    lines = ax.plot(time_ordered_cluster_centers[:, 0],
                    time_ordered_cluster_centers[:, 1],
                    time_ordered_cluster_centers[:, 2],
                    c='black',
                    alpha=0.5,
                    linewidth=3.0)
    for line in lines:
        add_arrow(line, color='black', size=15)
        
    ax.scatter(time_ordered_cluster_centers[:, 0],
               time_ordered_cluster_centers[:, 1],
               time_ordered_cluster_centers[:, 2],
               c=np.arange(len(clustered_correlation_timeseries_for_first_three_pcs.cluster_centers_[:, 0])), 
               alpha=1.0,
               cmap='YlGnBu',
               marker='o',
               s=400)
    ax.set_xlabel('PC1', fontsize=20)
    ax.set_ylabel('PC2', fontsize=20)
    ax.set_zlabel('PC3', fontsize=20)
    ax.set_title(k+" Clustered (Time+Space) Low-Dimensional Manifold", fontsize=30)
    subplot_idx += 1
    
    # Plot correlation timeseries for first 3 PCs in a 3d scatterplot
    ax = fig.add_subplot(len(pca_models), 5, subplot_idx, projection='3d')
    ax.plot(correlation_timeseries_for_first_three_pcs[0],
            correlation_timeseries_for_first_three_pcs[1],
            correlation_timeseries_for_first_three_pcs[2],
            c='black',
            alpha=0.2,
            linewidth=0.5)
    ax.scatter(correlation_timeseries_for_first_three_pcs[0],
               correlation_timeseries_for_first_three_pcs[1],
               correlation_timeseries_for_first_three_pcs[2],
               c=np.arange(len(correlation_timeseries_for_first_three_pcs[0])), 
               alpha=0.5,
               cmap='YlGnBu',
               marker='o')
    ax.set_xlabel('PC1', fontsize=20)
    ax.set_ylabel('PC2', fontsize=20)
    ax.set_zlabel('PC3', fontsize=20)
    ax.set_title(k+" Low Dimensional Manifold", fontsize=30)
    subplot_idx += 1
    
    # Plot correlation timeseries for each PC separately
    cm = plt.get_cmap('YlGnBu') 
    cNorm = colors.Normalize(vmin=0, vmax=len(correlation_timeseries_for_first_three_pcs[0]))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    colorVals = scalarMap.to_rgba(np.arange(len(correlation_timeseries_for_first_three_pcs[0])))
    
    ax = fig.add_subplot(len(pca_models), 5, subplot_idx)
    ax.plot(correlation_timeseries_for_first_three_pcs[0])#, color=colorVals)
    ax.set_xlabel('Time', fontsize=20)
    ax.set_ylabel('PC1 Correlation', fontsize=20)
    ax.set_title("{0} PC1".format(k), fontsize=30)
    subplot_idx += 1
    
    ax = fig.add_subplot(len(pca_models), 5, subplot_idx)
    ax.plot(correlation_timeseries_for_first_three_pcs[1])#, color=colorVals)
    ax.set_xlabel('Time', fontsize=20)    
    ax.set_ylabel('PC2 Correlation', fontsize=20)
    ax.set_title("{0} PC2".format(k), fontsize=30)
    subplot_idx += 1
    
    ax = fig.add_subplot(len(pca_models), 5, subplot_idx)
    ax.plot(correlation_timeseries_for_first_three_pcs[1])#, color=colorVals)
    ax.set_xlabel('Time', fontsize=20)
    ax.set_ylabel('PC3 Correlation', fontsize=20)
    ax.set_title("{0} PC3".format(k), fontsize=30)
    subplot_idx += 1

plt.savefig('output/principal_component_analysis/low_dimensional_manifold_traversal.png')
fig.close()

Compute correlations between correlation timeseries of PCs of different modalities.

In [ ]:
correlations_between_pcs_of_different_modalities = {}

for (k_a, pca_model_a) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, pca_models):
    for (k_b, pca_model_b) in zip(all_subjects_all_trials_connectome_upper_triangular_flattened, pca_models):
        for a_i in range(0, 3):
            for b_j in range(0, 3):
                
                # Extract PC i from modality a and PC j from modality b
                component_i = pca_model_a.components_[a_i]
                component_j = pca_model_b.components_[b_j]
                
                # Compute correlation timeseries of PC i in modality a and PC j in modality b
                connectome_timeseries_a = all_subjects_all_trials_connectome_upper_triangular_flattened[k_a]
                connectome_timeseries_b = all_subjects_all_trials_connectome_upper_triangular_flattened[k_b]
                
                correlation_timeseries_for_component_i_in_a = [np.corrcoef(component_i, time_pt)[0, 1] for time_pt in connectome_timeseries_a]
                correlation_timeseries_for_component_j_in_b = [np.corrcoef(component_j, time_pt)[0, 1] for time_pt in connectome_timeseries_b]

                # Compute correlation between correlation timeseries
                corr = np.corrcoef(correlation_timeseries_for_component_i_in_a, correlation_timeseries_for_component_j_in_b)[0, 1]                
                correlations_between_pcs_of_different_modalities["{0}-{1} <-> {2}-{3}".format(k_a, a_i, k_b, b_j)] = corr
                print("{0}-{1} <-> {2}-{3} = {4}".format(k_a, a_i, k_b, b_j, corr))

In [ ]:
labels = ["{0}-{1}".format(k, i) for k in all_subjects_all_trials_connectome_upper_triangular_flattened for i in range(0, 3)]
N = len(labels)
cross_modality_pc_correlation_matrix = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        lookup_key = "{0} <-> {1}".format(labels[i], labels[j])
        cross_modality_pc_correlation_matrix[i][j] = correlations_between_pcs_of_different_modalities[lookup_key]

In [ ]:
f = plt.figure(figsize=(15, 10))

plt.imshow(cross_modality_pc_correlation_matrix, cmap='gist_heat')
plt.title("Cross Modality Correlations of Principal Component Correlation Timeseries").set_position([.5, 1.3])
plt.axes().xaxis.set_ticks_position('top')
plt.xticks(range(len(labels)), labels, rotation='vertical')
plt.yticks(range(len(labels)), labels)
plt.colorbar()

f.savefig('output/principal_component_analysis/cross_modality_correlations_of_pc_timeseries.png')
plt.close()